In [1]:
# Commandes permettant d'éviter de relancer le notebook à chaque mise à jour de script 
%load_ext autoreload
%autoreload 2

In [2]:
import requests, zipfile, json, io
import pandas as pd
import sys
sys.path.insert(0,r"L:\6. Autre\projet_citycomp\data")
from data import read_data_from_local_drive, load_cities_geo_info_df, data_enrichment, \
                read_meta_data_from_local_drive, load_city_election_first_round, formatting_metropoles_votes, \
                load_city_election_second_round, votes_per_city, election_first_round_formatting,cod_geo_result_election, \
                election_second_round_formatting, concat_first_second_round, election_per_city_dataset_final,load_presidential_election, \
                cod_geo_result_presidential_election, presidential_election_formatting, download_data_housing_insee, housing_data_preproc, housing_cat_data_per_city, \
                housing_construction_date_df,nb_rooms_bucket
import numpy as np
import os
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import geopy.distance
import plotly.graph_objects as go
import calendar
from urllib.error import HTTPError


In [21]:
class Cityhall_election:
    def __init__(self, folder, source_file_name):
        self.data = pd.read_parquet(os.path.join(folder, source_file_name))
    def __str__(self):
        return "Cityhall_election"
    def load_data(self):
        df = self.data
        return df

cityhall_elec = Cityhall_election(r"L:\6. Autre\projet_citycomp\app\processed_data","2020_gouv_data_city_hall_elections.parquet.gzip")
cityhall_data = cityhall_elec.load_data()

ref_nuance = pd.read_csv("codes_nuances_2020.csv", sep=";")
ref_nuance

,Bloc,CodNua,LibNua,Definition,NumOrdNua
0,EXG,LEXG,Extrême gauche,Lutte ouvrière; Nouveau Parti Anticapitaliste;...,1
1,GAU,LCOM,Parti communiste français,Listes du Parti communiste français.,2
2,GAU,LFI,La France insoumise,Listes de La France insoumise.,3
3,GAU,LSOC,Parti socialiste,Listes du Parti socialiste.,4
4,GAU,LRDG,Parti radical de gauche,Listes du Parti radical de gauche.,5
5,GAU,LDVG,Divers gauche,Listes d'union investies par plusieurs partis ...,6
6,GAU,LUG,Union de la gauche,Listes soutenues par un parti de gauche ayant ...,7
7,GAU,LVEC,Europe Ecologie-Les Verts,Listes d'Europe Ecologie-Les Verts.,8
8,AUT,LECO,Ecologiste,Union des démocrates et des écologistes; Allia...,9
9,AUT,LDIV,Divers,Listes inclassables; Union Populaire Républica...,10
